In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
import csv
import tensorflow as tf
import nltk 

import gensim as gs

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from tensorflow.contrib import rnn

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

nltk.download('punkt')

df1 = pd.read_csv('HillaryTrumpTrainWithTags.csv')
df2 = pd.read_csv('TrumpHillaryTest.csv')
"""
df2 = pd.read_table('glove.twitter.27B.25d.txt', sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

def vectorize(w):
  return df2.loc[w].as_matrix()
"""
  

[nltk_data] Downloading package punkt to /Users/Alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'\ndf2 = pd.read_table(\'glove.twitter.27B.25d.txt\', sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)\n\ndef vectorize(w):\n  return df2.loc[w].as_matrix()\n'

In [2]:
def handleToNumber(handle):
    if handle == "realDonaldTrump":
        return 0
    else:
        return 1

In [3]:
df1['tokenized_tweets'] = df1.apply(lambda row: nltk.word_tokenize(row['tweet']), axis=1)
df1['labels'] = df1.apply(lambda row: handleToNumber(row['handle']), axis=1)
df2['tokenized_tweets'] = df2.apply(lambda row: nltk.word_tokenize(row['tweet']), axis=1)

In [4]:
tweets = df1['tokenized_tweets']
tweets_test = df2['tokenized_tweets']

In [5]:
length = []
_ = df1.apply(lambda row: length.append(len(row['tokenized_tweets'])), axis=1)
max_length = max(length)

In [6]:
max_length

54

In [7]:
vectorizer = gs.models.Word2Vec(df1['tokenized_tweets'].append(df2['tokenized_tweets']) , min_count=1, size = 25, iter = 10)

In [8]:
train_matrix = []
test_matrix = []

for i, tweet in enumerate(tweets):
    #print(type(string))
    train_matrix.append(vectorizer.wv[tweet])
    
for i, test_tweet in enumerate(tweets_test):
    test_matrix.append(vectorizer.wv[test_tweet])

In [9]:
# np.array(train_matrix).reshape(len(train_matrix),1)

In [10]:
train_matrix = np.array(train_matrix)
train_matrix[0]

test_matrix = np.array(test_matrix)
test_matrix[0]

array([[-2.75301766e-02,  2.37620771e-02,  7.10224882e-02,
         3.16517174e-01, -9.11614001e-02, -1.87514365e-01,
        -2.12408602e-02,  3.19370687e-01,  1.23226792e-01,
        -1.45338133e-01,  2.23934904e-01,  4.46902633e-01,
         3.74189205e-02,  3.17429677e-02,  6.97335154e-02,
        -2.04048201e-01,  5.98485656e-02,  5.37247695e-02,
        -2.51924455e-01,  3.35068792e-01,  4.51279022e-02,
         1.64801329e-01, -1.90304946e-02, -9.68852954e-04,
         1.72028374e-02],
       [ 3.77993196e-01, -1.52674508e+00, -7.89127946e-01,
         1.53935814e+00, -1.67936459e-01, -8.93275142e-01,
        -9.37681437e-01,  1.18657768e+00,  1.30125380e+00,
        -1.11688459e+00,  1.58860826e+00,  2.10358500e+00,
        -2.74104178e-01,  4.88766819e-01, -2.98717394e-02,
        -1.92562401e-01,  3.79370004e-01,  2.40486592e-01,
         2.89743003e-02,  1.96531606e+00,  1.46861091e-01,
        -5.35180867e-02,  1.27491355e+00, -2.20037985e+00,
         4.82255369e-01],
    

In [11]:
type(train_matrix)

numpy.ndarray

In [12]:
train_matrix.shape

(5000,)

In [13]:
train_matrix[0].shape

(23, 25)

In [14]:
twit = train_matrix[4999]

In [15]:
def pad(twits):
    """
    Does the padding as many times as required
    """
    l = len(twits)
    d= twits.shape[1]
    padding = np.zeros(d)
    
    for i in range(0,max_length-l):
        twits = np.append(twits,padding)
        
    return twits.reshape(max_length,d)

In [16]:
def pad_matrix(t_matrix):
    """
    Loops through all the vectorized tweets and calls the pad function on each of them.
    """
    
    padded_matrix =[]
    for i,vectorized_tweet in enumerate(t_matrix):
         padded_matrix.append(pad(vectorized_tweet))
    padded_matrix = np.array(padded_matrix)
    
    return padded_matrix

In [17]:
padded_train_matrix = pad_matrix(train_matrix)
padded_test_matrix = pad_matrix(test_matrix)

In [18]:
padded_train_matrix.shape

(5000, 54, 25)

In [19]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [20]:
labels = df1['labels']
labels_vector = []

for i, label in enumerate(labels):
    labels_vector.append([label, 1 - label])
labels_vector = np.array(labels_vector)

In [ ]:
tf.reset_default_graph()
#define constants
#unrolled through 28 time steps
time_steps=54
#hidden LSTM units
num_units=1024
#rows of 28 pixels
n_input=25
#learning rate for adam
learning_rate=0.01
#mnist is meant to be classified in 10 classes(0-9).
n_classes=2
#size of batch
batch_size=128

#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,n_input])
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

#processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
input=tf.unstack(x ,time_steps,1)

#defining the network
lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias
test=tf.nn.softmax(prediction)

#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

#initialize variables
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    iter=1
    while iter<500:
        batch_x,batch_y=next_batch(batch_size, padded_train_matrix, labels_vector)
        sess.run(opt, feed_dict={x: batch_x, y: batch_y})

        if iter %10==0:
#             acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
#             los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
# #             pred=sess.run(test, feed_dict={x:batch_x,y:batch_y})
# #             print(pred)
# #             print(batch_y)
            print("For iter ",iter)
#             print("Accuracy ",acc)
#             print("Loss ",los)
#             print("__________________")

        iter=iter+1
    pred=sess.run(test, feed_dict={x:padded_test_matrix})
    result = np.array(pred, dtype=np.float)
print("Done")

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

For iter  10
For iter  20


In [ ]:
result.shape
a = np.zeros(shape=(1444,3))
for i, res in enumerate(result):
    a[i] = [i, result[i][1], result[i][0]]

In [ ]:
np.set_printoptions(threshold=np.nan)
np.set_printoptions(suppress=True)
np.savetxt('test.out', a, delimiter=',', fmt='%1.19f')   # X is an array